In [1]:
import pickle
from pandas import read_csv,DataFrame,get_dummies
Test = read_csv("./test_data.csv")

### Initialize Some Columns to 0 : A lot of work was tested here but none of my ideas improves the score

In [2]:
Scored_Data = DataFrame()
No_errors = ['Expenditure','Status','Tag_3','PCVNumber',
             'T_Number','Tag_2','Sex',
             'Release_Admiss_Notes', 'Date_Caught',
             'TurtleCharacteristics','Date','Date_Release','Researcher','Species',
             'SpecialRemarks','Lost_Tags','CCW_cm','CCL_cm','Weight_Kg']
for Column in No_errors:
    Temp = DataFrame()
    Temp['ID'] = Test['Rescue_ID'] + ' x ' + Column
    Temp['error'] = 0
    Scored_Data = Scored_Data.append(Temp)
Scored_Data.head()

,ID,error
0,2011_RE_0001 x Expenditure,0
1,2011_RE_0002 x Expenditure,0
2,2011_RE_0003 x Expenditure,0
3,2011_RE_0004 x Expenditure,0
4,2011_RE_0005 x Expenditure,0


In [3]:
import numpy as np
from pandas import read_csv,to_datetime, DataFrame, pivot, get_dummies

# Train Data
Variable_Description = read_csv("./variable_definitions.csv",encoding='latin1')
Dirty_Data = read_csv("./dirty_data.csv",encoding='latin1')
print('Train rows :' ,Dirty_Data.shape[0])
# Test Data
Test_Data = read_csv('./test_data.csv')
print('Test rows :',Test_Data.shape[0])

Train rows : 4292
Test rows : 1362


### Tag 2 subsequent with Tag 1 ? 

### CaptureSite and ForagingGround
#### Here we take the most frequent ForagingGround for each CaptureSite and consider the others as erros, this could be done with an machine algorithm but a if then rule works

In [4]:
ForagingGround_Errors = Test_Data[['CaptureSite','ForagingGround']].append(Dirty_Data[['CaptureSite','ForagingGround']])

CaptureSite_to_ForagingGround = dict()
for site in ForagingGround_Errors.CaptureSite.unique():
    CaptureSite_to_ForagingGround[site] = ForagingGround_Errors.loc[ForagingGround_Errors.CaptureSite == site,'ForagingGround'].value_counts() \
                                          .reset_index()['index'][0]

Scored_Data = Scored_Data.loc[Scored_Data.ID.map(lambda x : 'ForagingGround' not in x)]

Temp = DataFrame()
Temp['ID'] = Test_Data['Rescue_ID'] + ' x ' + 'ForagingGround'
Temp['error'] = 0
for idx, row in Test_Data.iterrows():
    if (row['ForagingGround'] != CaptureSite_to_ForagingGround[row['CaptureSite']]):
        Temp.loc[Temp['ID'] == row['Rescue_ID'] + ' x ' + 'ForagingGround', 'error'] = 1
Scored_Data = Scored_Data.append(Temp)
Scored_Data.error.sum()

126

### Compare to last numerical value for each tag

### Load and make predictions (taking predictions instead of probability increase the score)

In [5]:
Cat_Errors = ['CaptureSite','LandingSite','ReleaseSite',
              'CaptureMethod','Fisher','Tag_1']

svc_classif = pickle.load(open('./Cat_model.p','rb'))
vect_space = pickle.load(open('./vect_space.p','rb'))

Temp = Test[['Rescue_ID'] + Cat_Errors].fillna("NA").set_index('Rescue_ID')

Test_set = get_dummies(Temp)
New_features = [x for x in Test_set if x not in vect_space]
Old_features = [x for x in vect_space if x not in Test_set]
for o in Old_features:
    Test_set[o] = 0


Y_test_pred = DataFrame(svc_classif.predict(Test_set[vect_space]),
                        columns=Cat_Errors,index=Temp.index)

for Column in Cat_Errors:
    Temp = Y_test_pred[Column].reset_index()
    Temp['ID'] = Temp['Rescue_ID'] + ' x ' + Column
    Temp['error'] = Temp[Column]
    Scored_Data = Scored_Data.append(Temp[['ID','error']])
Scored_Data.tail()

,ID,error
1357,2011_RE_1361 x Tag_1,0
1358,2011_RE_1362 x Tag_1,0
1359,2011_RE_1363 x Tag_1,0
1360,2011_RE_1364 x Tag_1,0
1361,2011_RE_1365 x Tag_1,0


### Write Submission File

In [6]:
Scored_Data.to_csv('./submission_v0.csv',index=False)